In [34]:
import math
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
%run ./2-ImplementationFactor.ipynb
%run ./3-ImplementationPGM.ipynb
%run ./4-ImplementationBP.ipynb

./img/3/graph.html
./img/3/graph.html


In [35]:
def delta_funct(S_ki, t_i):
    if (t_i == np.min(S_ki)):
        return 1
    return 0

In [36]:
def R_i(t_i, r_i, lambda_i):
    if (t_i > r_i):
        return 0
    result = lambda_i*np.exp(-lambda_i*(r_i - t_i))
    return result

In [37]:
def S_ij(s_ij, t_i, r_i, lambda_i, time_range, contact_range_ij):
    if (t_i <= s_ij < r_i):
            temp = np.where((contact_range_ij < s_ij) & (contact_range_ij >= t_i))
            return (lambda_i)*(1-lambda_i)**(len(temp))
    elif (s_ij ==9):
        index_larger_than_t_i = np.where(contact_range_ij >= t_i)
        return (1-lambda_i)**(len(index_larger_than_t_i))
    else:
        return 0

In [38]:
#debug function
def Find_Zero(delt, R, S):
    if (delt == 0):
        print("delt = 0")
    if (R == 0):
        print("R = 0")
    if (S == 0):
        print("S = 0")

def phi(t_i,r_i, s):
    delt = delta_funct([s[1]],t_i)
    R = R_i(t_i, r_i, 0.3)
    S = S_ij(s[0], t_i, r_i, 0.3, "nothing", np.array([0,6,9]))
    #Find_Zero(delt, R, S)
    return delt*R*S


In [39]:
#phi(t_i,r_i,(s_ij,s_ji))

In [40]:
phi(0, 4, (3,0))

0.018975235350468734

In [41]:
#inference using BP

In [42]:
variables_i = ["t_i","r_i","s_ij:s_ji"]
variables_j = ["t_j", "r_j","s_ij:s_ji"]

In [43]:
def map_digit_to_2d_array(n):
    x = n % 10
    y = n // 10
    return [x, y]

In [44]:
for x in range(100):
    print( map_digit_to_2d_array(x))

[0, 0]
[1, 0]
[2, 0]
[3, 0]
[4, 0]
[5, 0]
[6, 0]
[7, 0]
[8, 0]
[9, 0]
[0, 1]
[1, 1]
[2, 1]
[3, 1]
[4, 1]
[5, 1]
[6, 1]
[7, 1]
[8, 1]
[9, 1]
[0, 2]
[1, 2]
[2, 2]
[3, 2]
[4, 2]
[5, 2]
[6, 2]
[7, 2]
[8, 2]
[9, 2]
[0, 3]
[1, 3]
[2, 3]
[3, 3]
[4, 3]
[5, 3]
[6, 3]
[7, 3]
[8, 3]
[9, 3]
[0, 4]
[1, 4]
[2, 4]
[3, 4]
[4, 4]
[5, 4]
[6, 4]
[7, 4]
[8, 4]
[9, 4]
[0, 5]
[1, 5]
[2, 5]
[3, 5]
[4, 5]
[5, 5]
[6, 5]
[7, 5]
[8, 5]
[9, 5]
[0, 6]
[1, 6]
[2, 6]
[3, 6]
[4, 6]
[5, 6]
[6, 6]
[7, 6]
[8, 6]
[9, 6]
[0, 7]
[1, 7]
[2, 7]
[3, 7]
[4, 7]
[5, 7]
[6, 7]
[7, 7]
[8, 7]
[9, 7]
[0, 8]
[1, 8]
[2, 8]
[3, 8]
[4, 8]
[5, 8]
[6, 8]
[7, 8]
[8, 8]
[9, 8]
[0, 9]
[1, 9]
[2, 9]
[3, 9]
[4, 9]
[5, 9]
[6, 9]
[7, 9]
[8, 9]
[9, 9]


In [45]:
shape = (10,10,100)
matrix_phi_i = np.zeros(shape=(10,10,100))
matrix_phi_j = np.zeros(shape=(10,10,100))
for index in np.ndindex(shape):
    matrix_phi_i[index[0], index[1], index[2]] = phi(index[0], index[1], map_digit_to_2d_array(index[2]))
for index in np.ndindex(shape):
    matrix_phi_j[index[0], index[1], index[2]] = phi(index[0], index[1], map_digit_to_2d_array(index[2]))

In [46]:
phi_i = factor(variables_i, matrix_phi_i)
phi_j = factor(variables_j, matrix_phi_j)

In [47]:
graph = factor_graph()
graph.add_factor_node("phi_i", phi_i)
graph.add_factor_node("phi_j", phi_j)

In [48]:
bp = belief_propagation(graph)
bp_result = bp.belief("t_i").get_distribution()

In [49]:
bp_result

array([0.15943275, 0.15307027, 0.14475433, 0.13400012, 0.1202956 ,
       0.10319189, 0.08249319, 0.05862558, 0.03331813, 0.01081813])

In [50]:
plot_factor_graph(graph)

./img/3/graph.html


In [51]:
#exact calculation of marginals

In [52]:
t_i_array = [0]*10
for t_i in range(10):
    sum = 0
    for r_i in range(10):
        for t_j in range(10):
            for r_j in range(10):
                for s_ij in range(100):
                    sum += phi(t_i, r_i, map_digit_to_2d_array(s_ij))*phi(t_j, r_j, map_digit_to_2d_array(s_ij))
    t_i_array[t_i] = sum

In [53]:
sum_Z = np.sum(t_i_array)

In [54]:
t_i_array

[0.6499257998216534,
 0.6239892209168375,
 0.5900893926245488,
 0.5462499774828933,
 0.49038366944169237,
 0.420660559003195,
 0.33628256076482377,
 0.23898651825065323,
 0.13582099359953126,
 0.04409999999999999]

In [55]:
marginal_t_i = t_i_array/sum_Z
marginal_t_i

array([0.15943275, 0.15307027, 0.14475433, 0.13400012, 0.1202956 ,
       0.10319189, 0.08249319, 0.05862558, 0.03331813, 0.01081813])

In [56]:
bp_result

array([0.15943275, 0.15307027, 0.14475433, 0.13400012, 0.1202956 ,
       0.10319189, 0.08249319, 0.05862558, 0.03331813, 0.01081813])

In [57]:
factor_i = factor(["t_i", "t_j"], np.array([[0.5, 0.6], [0.7, 0.8]]))
factor_j = factor(["t_j"], np.array([0.5, 0.6]))

graph = factor_graph()
#graph.add_factor_node("factor_i", factor_i)
graph.add_factor_node("factor_j", factor_j)

bp = belief_propagation(graph)
bp_result = bp.belief("t_j").get_distribution()

In [58]:
bp_result

array([0.45454545, 0.54545455])

In [59]:
#exact calculation of marginals
exact = []
matrix =  np.array([[0.5, 0.6], [0.7, 0.8]])
for t_j in range(2):
        sum = 0
        sum += factor_i.get_distribution()[0,t_j]
        exact.append(sum)
sum_Z = np.sum(exact)
exact

[0.5, 0.6]

In [60]:
fac = factor_reduction(factor_i, "t_i", 0)

In [61]:
np.array(exact)/sum_Z

array([0.45454545, 0.54545455])